In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from google import genai
import os
load_dotenv()

True

In [2]:
#Creating the object of GEMINI Model 
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
# ask that question to the LLM
response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt
    )

In [3]:
class BlogState(TypedDict):
    title:str
    outline: str
    content: str

In [4]:
def create_outline(state: BlogState)->BlogState:
    #fetch title
    title = state['title']

    #call llm gen outline
    prompt = f'Generate an outline for a blog on the topic - {title}'
    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt
    )

    #update state
    state['outline'] = response
    return state


def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the following outline \n {outline}'

    response = client.models.generate_content(
        model="gemini-2.5-flash", 
        contents=prompt
    )

    state['content'] = response
    return state



In [5]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)


#edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)



workflow = graph.compile()



In [6]:
initial_State = {'title': 'Rise of AI in India'}
final_state  = workflow.invoke(initial_State)
print(final_state)

{'title': 'Rise of AI in India', 'outline': GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""Here's a comprehensive outline for a blog post on "The Rise of AI in India," designed to be engaging, informative, and well-structured.

---

## Blog Title Options:

*   **India's AI Revolution: Powering the Future of a Nation**
*   **The AI Ascent: Charting the Rise of Artificial Intelligence in India**
*   **From Niche to National Imperative: The Rise of AI in India**

---

## Blog Outline: The AI Ascent: Charting the Rise of Artificial Intelligence in India

**I. Introduction: The Dawn of a New Era**

*   **A. Hook:** Start with a vivid statement about the global AI wave and its transformative power.
    *   *Example:* "From smart assistants to self-driving cars, Artificial Intelligence is no longer sci-fi; it's reshaping our world. And at the heart of this global revolut

In [8]:
final_state['outline'].text

'Here\'s a comprehensive outline for a blog post on "The Rise of AI in India," designed to be engaging, informative, and well-structured.\n\n---\n\n## Blog Title Options:\n\n*   **India\'s AI Revolution: Powering the Future of a Nation**\n*   **The AI Ascent: Charting the Rise of Artificial Intelligence in India**\n*   **From Niche to National Imperative: The Rise of AI in India**\n\n---\n\n## Blog Outline: The AI Ascent: Charting the Rise of Artificial Intelligence in India\n\n**I. Introduction: The Dawn of a New Era**\n\n*   **A. Hook:** Start with a vivid statement about the global AI wave and its transformative power.\n    *   *Example:* "From smart assistants to self-driving cars, Artificial Intelligence is no longer sci-fi; it\'s reshaping our world. And at the heart of this global revolution, one nation is emerging as a formidable force: India."\n*   **B. Context Setting:** Briefly introduce India\'s unique position – a young, tech-savvy population, massive digital penetration, 

In [ ]:
final_state